In [322]:
import pandas as pd 
import numpy as np
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [323]:
info_df = pd.read_csv(r'LA_districts.csv') #import financial/teacher/pupil info per district of LA
grades_df = pd.read_csv(r'performance_summary.csv') #import grading of each district of LA

# Project Goals
The team, **Nick Ouellet and Garrett Gilliom**, will likely be investigating data sets related to Louisiana public school districts that measure student, faculty, and staff demographics, performance, district finances, and more. So far, the team has come across two data sets that measure different variables and attributes of the Louisiana school districts across the same year that appear to be well-populated. The website in which the analysis is located can be found [here](https://nickouellet.github.io)


### Collaboration Plan
The team plans to meet once a week to work on this project to finalize our project vision, determine objectives, and assist each other where necessary. We will also be communicating thoroughly over text messages to keep each other updated with task progress and will be using Google Docs to keep an ongoing list of to-do items that’s accessible by each of us at all times. The team has also set up a shared Github repository to store, coordinate, and update our code.

## Table 1: 
This first table is from the U.S. Department of Education National Center for Education Statistics Common Code of Data (CCD). Their website allows users to make specific queries to their database to get personalized data; the team opted to focus on Louisiana public school districts, each of which is its own observation and row, and included nearly all variable measurements to be included for each district, including diversity levels within the district, pupil/teacher ratio, total revenue from different sources, and expenditures. The team chose this data set because it provides data on many points and variables that may be relevant upon further investigation when answering questions such as, “What school districts are given the most funding?” “In what school districts are teachers compensated the most?” “How do expenditures align with compensation?” and “Is there any correlation between student demographics, including sex, gender, race, and/or ethnicity, and these measurements?” Furthermore, its source is reputable and trustable, being the U.S. Department of Education. However, the team also recognizes that the variables included in the requested dataset were chosen with inherent bias; it is the team’s goal to limit this bias as much as possible, and therefore will best attempt to objectively review what measurements are necessary or relevant in future milestones and project submissions.

In [324]:
info_df.head()

,Agency Name,State Name [District] Latest available year,State Name [District] 2016-17,Agency Name [District] 2016-17,Agency Type [District] 2016-17,Total Students All Grades (Includes AE) [District] 2016-17,Male Students [District] 2016-17,Female Students [District] 2016-17,American Indian/Alaska Native - male [District] 2016-17,American Indian/Alaska Native - female [District] 2016-17,Asian or Asian/Pacific Islander - male [District] 2016-17,Asian or Asian/Pacific Islander - female [District] 2016-17,Hispanic - male [District] 2016-17,Hispanic - female [District] 2016-17,Black or African American - male [District] 2016-17,Black or African American - female [District] 2016-17,White - male [District] 2016-17,White - female [District] 2016-17,Nat. Hawaiian or Other Pacific Isl. - male [District] 2016-17,Nat. Hawaiian or Other Pacific Isl. - female [District] 2016-17,Two or More Races - male [District] 2016-17,Two or More Races - female [District] 2016-17,Full-Time Equivalent (FTE) Teachers [District] 2016-17,Pupil/Teacher Ratio [District] 2016-17,Kindergarten Teachers [District] 2016-17,Elementary Teachers [District] 2016-17,Secondary Teachers [District] 2016-17,Total Staff [District] 2016-17,Total General Revenue (TOTALREV) [District Finance] 2016-17,Total Revenue - Federal Sources (TFEDREV) [District Finance] 2016-17,Total Revenue - State Sources (TSTREV) [District Finance] 2016-17,Total Revenue - Local Sources (TLOCREV) [District Finance] 2016-17,Total Revenue (TOTALREV) per Pupil (V33) [District Finance] 2016-17,Instruction Expenditures - Total (E13) [District Finance] 2016-17,Salary - Instruction Expenditures (Z33) [District Finance] 2016-17
0,A.E. PHILLIPS LABORATORY SCHOOL,Louisiana,LOUISIANA,A.E. Phillips Laboratory School,8-Other education agencies,394,194.0,195,–,–,5,5,3,1,41,41,148,149,–,–,–,1,22.00,17.91,2.50,18.50,1.00,25.00,–,–,–,–,"=""0""",–,–
1,ABRAMSON SCI ACADEMY,LOUISIANA,LOUISIANA,RSD-Collegiate Academies,7-Independent Charter District,561,307.0,253,1,–,21,21,5,2,268,221,8,5,1,2,3,3,36.99,15.17,†,†,36.99,83.74,8252000,1397000,3605000,3250000,14709,3473000,2588000
2,ACADIA PARISH,Louisiana,LOUISIANA,Acadia Parish,1-Regular local school district that is NOT a ...,9839,5050.0,4787,9,9,9,5,149,115,1339,1212,3347,3233,–,–,198,214,548.94,17.92,39.00,322.36,163.58,1119.35,95051000,15149000,53342000,26560000,9661,54183000,35390000
3,ACADIANA RENAISSANCE CHARTER ACADEMY,LOUISIANA,LOUISIANA,Acadiana Renaissance Charter Academy,7-Independent Charter District,887,430.0,454,1,3,3,4,20,15,39,46,367,388,–,–,1,–,53.08,16.71,6.95,27.16,18.97,86.36,8941000,433000,3525000,4983000,10080,4328000,3089000
4,ADVANTAGE CHARTER ACADEMY,LOUISIANA,LOUISIANA,Advantage Charter Academy,7-Independent Charter District,553,279.0,267,–,1,–,–,8,6,268,253,7,10,–,–,–,–,34.30,16.12,5.27,23.03,6.00,67.63,6703000,753000,2952000,2998000,12121,2891000,1776000


This data table not only includes parish (equivalent to districts) information, but information about each school. Because we only wish to study parish data, we will drop the rest of the columns.

In [325]:
def check_parish(entry):
    if re.search(r'PARISH',entry) == None:
        return "not a parish"
    return entry

In [326]:
info_df['Agency Name'] = info_df['Agency Name'].apply(check_parish) #mark all entries that are no parishes
info_df = info_df[info_df["Agency Name"] != "not a parish"]
print(len(info_df)) #we know we have succedded because there are 64 entries and 64 parishes in LA

64


We are only studying Louisiana schools, so will will drop the columns that give us redundant data (namely 'State Name [District] Latest available year' and 'State Name [District] 2016-17'). We will also drop 'Agency Name [District]' column because we already are representing that info and 'Agency Type [District] 2016-17' because we know we are only dealing with districts.

In [327]:
info_df.drop(columns = ["State Name [District] Latest available year", "State Name [District] 2016-17", "Agency Name [District] 2016-17","Agency Type [District] 2016-17"], inplace = True)
info_df = info_df.reset_index(drop=True) #reset the numberical index to be 0-63 to accomidate for dropped rows


Now we will rename certain columns to remove excess wording.

In [328]:
info_df = info_df.rename(columns = { "Agency Name": "Parish",
                                     "Total Students All Grades (Includes AE) [District] 2016-17":"Total Students",
                                     'Male Students [District] 2016-17': "Male Students",
                                     'Female Students [District] 2016-17':"Female Students", 
                                     'American Indian/Alaska Native - male [District] 2016-17': 'American Indian/Alaska Native - Male',
                                     "American Indian/Alaska Native - female [District] 2016-17": "American Indian/Alaska Native - Female",
                                     'Asian or Asian/Pacific Islander - male [District] 2016-17':"Asian or Asian/Pacific Islander - Male",
                                     'Asian or Asian/Pacific Islander - female [District] 2016-17': "Asian or Asian/Pacific Islander - Female",
                                     "Total General Revenue (TOTALREV) [District Finance] 2016-17": "Total Revenue",
                                     "Total Revenue - Federal Sources (TFEDREV) [District Finance] 2016-17": "Federal Revenue", 
                                     "Total Revenue - State Sources (TSTREV) [District Finance] 2016-17": "State Revenue", 
                                     "Total Revenue - Local Sources (TLOCREV) [District Finance] 2016-17": "Local Revenue",
                                     "Total Revenue (TOTALREV) per Pupil (V33) [District Finance] 2016-17": "Revenue per Pupil", 
                                     "Instruction Expenditures - Total (E13) [District Finance] 2016-17": "Instruction Expendeture", 
                                     "Salary - Instruction Expenditures (Z33) [District Finance] 2016-17": "Salary Expendeture", 
                                     'Hispanic - male [District] 2016-17': "Hispanic - Male",
                                    'Hispanic - female [District] 2016-17':"Hispanic - Female",
                                    'Black or African American - male [District] 2016-17':"Black or African American - Male",
                                    'Black or African American - female [District] 2016-17':"Black or African American - Female",
                                    'White - male [District] 2016-17':"White - Male",
                                    'White - female [District] 2016-17':"White - Female",
                                    'Nat. Hawaiian or Other Pacific Isl. - male [District] 2016-17':"Nat. Hawaiian or Other Pacific Isl. - Male",
                                    'Nat. Hawaiian or Other Pacific Isl. - female [District] 2016-17':"Nat. Hawaiian or Other Pacific Isl. - Female",
                                    'Two or More Races - male [District] 2016-17':"Two or More Races - Male",
                                    'Two or More Races - female [District] 2016-17':"Two or More Races - Female",
                                    'Full-Time Equivalent (FTE) Teachers [District] 2016-17':"Full-Time Teachers",
                                    'Pupil/Teacher Ratio [District] 2016-17':"Pupil/Teacher Ratio",
                                    'Kindergarten Teachers [District] 2016-17':"Kindergarten Teachers",
                                    'Elementary Teachers [District] 2016-17':"Elementary Teachers",
                                    'Secondary Teachers [District] 2016-17':"Secondary Teachers",
                                    'Total Staff [District] 2016-17':"Total Staff"
                                     })

Now, we will remove the "PARISH" values from each row entry under the Parish column and also return the entries as lower case.

In [329]:
def remove_Parish(entry):
    return re.sub(r" PARISH", r"", entry)

def get_lower(entry):
    return entry.lower()

In [330]:
info_df['Parish'] = info_df['Parish'].apply(remove_Parish)
info_df['Parish'] = info_df['Parish'].apply(get_lower)

Lets now remove all missing data and replace it with NaN.

In [331]:
def replace_empty_with_NAN(entry):
    entry = str(entry)
    if re.search("–", entry) != None:
        return np.nan
    elif re.search('="0"',entry) != None:
        return np.nan
    elif re.search('="0.00"', entry) != None:
        return np.nan
    return entry


In [332]:
for col in info_df.columns:
    info_df[col] = info_df[col].apply(replace_empty_with_NAN)

Checking the dtypes, we find that the columns are not properly formatted so we make the appropriate changes. 

In [333]:
info_df.dtypes

Parish                                          object
Total Students                                  object
Male Students                                   object
Female Students                                 object
American Indian/Alaska Native - Male            object
American Indian/Alaska Native - Female          object
Asian or Asian/Pacific Islander - Male          object
Asian or Asian/Pacific Islander - Female        object
Hispanic - Male                                 object
Hispanic - Female                               object
Black or African American - Male                object
Black or African American - Female              object
White - Male                                    object
White - Female                                  object
Nat. Hawaiian or Other Pacific Isl. - Male      object
Nat. Hawaiian or Other Pacific Isl. - Female    object
Two or More Races - Male                        object
Two or More Races - Female                      object
Full-Time 

In [334]:
lst_of_col = ["Total Students", "Male Students","Female Students", "American Indian/Alaska Native - Male", "American Indian/Alaska Native - Female","Asian or Asian/Pacific Islander - Male", "Asian or Asian/Pacific Islander - Female","Hispanic - Male", "Hispanic - Female",
"Black or African American - Male",
"Black or African American - Female",
"White - Male",
"White - Female",
"Nat. Hawaiian or Other Pacific Isl. - Male",
"Nat. Hawaiian or Other Pacific Isl. - Female",
"Two or More Races - Male",
"Two or More Races - Female", 
"Total Staff",
"Total Revenue",
"Federal Revenue",
"State Revenue",
"Local Revenue",
"Revenue per Pupil",
"Instruction Expendeture",
"Salary Expendeture",
"Full-Time Teachers",
"Pupil/Teacher Ratio",
"Kindergarten Teachers",
"Elementary Teachers",
"Secondary Teachers"]

for col in lst_of_col:
    info_df[col] = info_df[col].astype('float')

In [335]:
info_df.dtypes #much better

Parish                                           object
Total Students                                  float64
Male Students                                   float64
Female Students                                 float64
American Indian/Alaska Native - Male            float64
American Indian/Alaska Native - Female          float64
Asian or Asian/Pacific Islander - Male          float64
Asian or Asian/Pacific Islander - Female        float64
Hispanic - Male                                 float64
Hispanic - Female                               float64
Black or African American - Male                float64
Black or African American - Female              float64
White - Male                                    float64
White - Female                                  float64
Nat. Hawaiian or Other Pacific Isl. - Male      float64
Nat. Hawaiian or Other Pacific Isl. - Female    float64
Two or More Races - Male                        float64
Two or More Races - Female                      

In [336]:
info_df.head() #a look at the final dataframe after data has been tidied

,Parish,Total Students,Male Students,Female Students,American Indian/Alaska Native - Male,American Indian/Alaska Native - Female,Asian or Asian/Pacific Islander - Male,Asian or Asian/Pacific Islander - Female,Hispanic - Male,Hispanic - Female,Black or African American - Male,Black or African American - Female,White - Male,White - Female,Nat. Hawaiian or Other Pacific Isl. - Male,Nat. Hawaiian or Other Pacific Isl. - Female,Two or More Races - Male,Two or More Races - Female,Full-Time Teachers,Pupil/Teacher Ratio,Kindergarten Teachers,Elementary Teachers,Secondary Teachers,Total Staff,Total Revenue,Federal Revenue,State Revenue,Local Revenue,Revenue per Pupil,Instruction Expendeture,Salary Expendeture
0,acadia,9839.0,5050.0,4787.0,9.0,9.0,9.0,5.0,149.0,115.0,1339.0,1212.0,3347.0,3233.0,NaN,NaN,198.0,214.0,548.94,17.92,39.00,322.36,163.58,1119.35,95051000.0,15149000.0,53342000.0,26560000.0,9661.0,54183000.0,35390000.0
1,allen,4279.0,2167.0,2112.0,29.0,19.0,22.0,18.0,33.0,30.0,438.0,436.0,1581.0,1545.0,2.0,1.0,62.0,63.0,321.63,13.30,22.50,187.11,93.52,623.56,48581000.0,4348000.0,29928000.0,14305000.0,11353.0,26659000.0,17209000.0
2,ascension,21942.0,11398.0,10544.0,32.0,32.0,157.0,114.0,857.0,775.0,3474.0,3232.0,6649.0,6136.0,15.0,17.0,214.0,238.0,1339.28,16.38,44.99,852.30,414.69,2603.89,308842000.0,47048000.0,98359000.0,163435000.0,14075.0,158570000.0,84263000.0
3,assumption,3589.0,1845.0,1742.0,4.0,5.0,8.0,8.0,85.0,81.0,796.0,724.0,931.0,903.0,NaN,NaN,22.0,22.0,214.98,16.69,11.05,118.38,65.01,506.84,42026000.0,5487000.0,22872000.0,13667000.0,11710.0,21872000.0,13509000.0
4,avoyelles,5534.0,2892.0,2641.0,33.0,23.0,18.0,13.0,22.0,31.0,1387.0,1251.0,1340.0,1243.0,1.0,NaN,91.0,81.0,271.80,20.36,20.91,120.59,117.30,614.02,54910000.0,8976000.0,33037000.0,12897000.0,9922.0,29270000.0,15720000.0


___

## Table 2:
The second dataset comes from the Louisiana Department of Education and includes data relating to measurements of student performance, including a “Letter Grade,” Assessment Index/Index ACT, strength of diploma, cohort graduation rate index, and progress points. Each school district is, again, separated as their own observation, which marks school districts as the unit of observation across both this and the former dataset. Again, this dataset comes from a reputable source, being the Louisiana Department of Education; the data also recognizes that some of its measurements may not be too complete or are unreliable and marks values as such; however, nearly every cell is filled in, so little cleaning will need to be done otherwise. This dataset may provide insight into questions such as, “What school districts perform best in Louisiana?” “What performance-related measurements correlate with each other?” “What are poorly performing districts struggling with?” and “What might poorly performing districts attempt to focus on to improve?”


In [337]:
grades_df.head() #Livingston parish not reported due to flooding in '16-'17

,District,2017 Letter Grade,2017 Annual DPS,2016 Letter Grade,2016 Annual DPS,2017 Assessment Index\nGrades 3-8*,2017 Dropout Credit Accumulation Index*,2017 Assessment Index\nEnd-of-Course Exams*,2017 Assessment Index\nACT*,Strength of Diploma (Graduation Index) (2015-16 Cohort)*,Cohort Graduation Rate Index \n(Points Earned for Cohort Graduation Rate) \n(2015-16 Cohort)*,Cohort Graduation Rate (Actual Graduation Rate) (2015-16 Cohort)**,2017 Progress Points,2016 Assessment Index\nGrades 3-8*,2016 Dropout Credit Accumulation Index*,2016 Assessment Index\nEnd-of-Course Exams*,2016 Assessment Index\nACT*,Strength of Diploma (Graduation Index) (2014-15 Cohort)*,Cohort Graduation Rate Index \n(Points Earned for Cohort Graduation Rate) \n (2014-15 Cohort)*,Cohort Graduation Rate (Actual Graduation Rate) (2014-15 Cohort)**,2016 Progress Points,Unnamed: 21
0,Acadia Parish,B,90.0,B,97.3,78.8,137.4,70.5,90.6,93.4,122.6,86.3,3.3,81.6,139.8,69.6,87.2,89.8,120.2,85.1,10.0,NaN
1,Allen Parish,A,103.4,A,104.2,85.9,143.6,77.4,98.2,96.3,127.6,88.8,10.0,88.6,139.8,68.9,93.4,98.9,132.2,91.1,10.0,NaN
2,Ascension Parish,A,110.4,A,109.3,92.6,141.8,97.9,99.0,105.9,125.8,87.9,10.0,92.6,140.9,97.1,95.0,101.5,125.4,87.7,10.0,NaN
3,Assumption Parish,B,95.0,B,93.3,75.5,143.6,67.9,91.4,90.7,119.6,84.8,10.0,78.0,139.7,68.9,75.9,88.1,111,80.5,10.0,NaN
4,Avoyelles Parish,C,71.5,C,73.6,53.7,128.6,54.7,95.1,86.6,103.4,76.7,3.1,57.3,123,48.3,91.4,75.1,93.2,71.6,6.2,NaN


Rename 'District' column to 'Parish', and lower case all parish names as well as removing "Parish" from values.

In [338]:
def remove_parish(entry):
    return re.sub(r" parish", r"", entry)

grades_df = grades_df.rename(columns = {"District": "Parish"})
grades_df["Parish"] = grades_df["Parish"].apply(get_lower)
grades_df["Parish"] = grades_df["Parish"].apply(remove_parish) #important to run this command after the get_lower() command because get_parish is only looking for lower case 

In [339]:
grades_df.head()

,Parish,2017 Letter Grade,2017 Annual DPS,2016 Letter Grade,2016 Annual DPS,2017 Assessment Index\nGrades 3-8*,2017 Dropout Credit Accumulation Index*,2017 Assessment Index\nEnd-of-Course Exams*,2017 Assessment Index\nACT*,Strength of Diploma (Graduation Index) (2015-16 Cohort)*,Cohort Graduation Rate Index \n(Points Earned for Cohort Graduation Rate) \n(2015-16 Cohort)*,Cohort Graduation Rate (Actual Graduation Rate) (2015-16 Cohort)**,2017 Progress Points,2016 Assessment Index\nGrades 3-8*,2016 Dropout Credit Accumulation Index*,2016 Assessment Index\nEnd-of-Course Exams*,2016 Assessment Index\nACT*,Strength of Diploma (Graduation Index) (2014-15 Cohort)*,Cohort Graduation Rate Index \n(Points Earned for Cohort Graduation Rate) \n (2014-15 Cohort)*,Cohort Graduation Rate (Actual Graduation Rate) (2014-15 Cohort)**,2016 Progress Points,Unnamed: 21
0,acadia,B,90.0,B,97.3,78.8,137.4,70.5,90.6,93.4,122.6,86.3,3.3,81.6,139.8,69.6,87.2,89.8,120.2,85.1,10.0,NaN
1,allen,A,103.4,A,104.2,85.9,143.6,77.4,98.2,96.3,127.6,88.8,10.0,88.6,139.8,68.9,93.4,98.9,132.2,91.1,10.0,NaN
2,ascension,A,110.4,A,109.3,92.6,141.8,97.9,99.0,105.9,125.8,87.9,10.0,92.6,140.9,97.1,95.0,101.5,125.4,87.7,10.0,NaN
3,assumption,B,95.0,B,93.3,75.5,143.6,67.9,91.4,90.7,119.6,84.8,10.0,78.0,139.7,68.9,75.9,88.1,111,80.5,10.0,NaN
4,avoyelles,C,71.5,C,73.6,53.7,128.6,54.7,95.1,86.6,103.4,76.7,3.1,57.3,123,48.3,91.4,75.1,93.2,71.6,6.2,NaN


*This is the tidying that we will do for now. As the project progresses, we will make this Dataframe more pandas function friendly.*

___
## Closing Thoughts and Final Goals:
The team is also interested in what relationships the two datasets have with each other, considering they cover different aspects of the same units of observations, those being demographic measurements and performance measurements. Questions such as, “What are the demographics of schools that perform best/worst?” “How do compensation rates compare between well and poorly performing schools?” “How relevant is the pupil/teacher ratio in academic performance?” and “Does school size have any impact on graduation rate or academic performance?” These are the questions the team is most interested in: the ones that relate to both datasets and will require the merging and/or comparing of the two, rather than just looking at each individually. Eventually, the team would like to create a model based on the data from these tables to predict the test scores of districts around the country based on their information from the U.S. Department of Education National Center for Education Statistics Common Code of Data (CCD). This would, of course, requires more datasets to be parsed and tidied – which would be done once our initial exploratory analysis is completed.